## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [2]:
df = pd.read_csv("ab_data.csv")
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the below cell to find the number of rows in the dataset.

In [3]:
df.shape

(294478, 5)

c. The number of unique users in the dataset.

In [4]:
num_users = df[['user_id']].user_id.nunique()
num_users

290584

d. The proportion of users converted.

In [5]:
users_converted = df[df['converted'] == 1].user_id.nunique()

proportion_converted = users_converted / num_users
proportion_converted

0.12104245244060237

e. The number of times the `new_page` and `treatment` don't line up.

In [6]:
# create boolean masks:
treatmentY_and_newPageN = (df['group'] == 'treatment') & (df['landing_page'] != 'new_page')
treatmentN_and_newPageY = (df['group'] != 'treatment') & (df['landing_page'] == 'new_page')
num_notAligned = df[treatmentY_and_newPageN].shape[0] + df[treatmentN_and_newPageY].shape[0]
num_notAligned

3893

f. Do any of the rows have missing values?

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


* No, none of the rows have missing values.

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [8]:
# # create boolean masks:
treatmentY_and_newPageY = (df['group'] == 'treatment') & (df['landing_page'] == 'new_page')
controlY_and_oldPageY = (df['group'] == 'control') & (df['landing_page'] == 'old_page')
df2 = df[(treatmentY_and_newPageY) | (controlY_and_oldPageY)]    # returns a df with only aligned rows

# NOTE - to return a df with only unaligned rows:
# df[~((treatmentY_and_newPageY) | (controlY_and_oldPageY))]

In [9]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

# # A multi-line approach:
# cond1 = (df2['group'] == 'treatment')
# cond2 = (df2['landing_page'] == 'new_page')
# df2[( cond1 == cond2) == False]

0

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [10]:
num_users_df2 = df2[['user_id']].user_id.nunique()
num_users_df2

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [11]:
df2[df2['user_id'].duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


* Return just the user_id value:

In [12]:
df2[df2['user_id'].duplicated()].iloc[0]['user_id']

773192

c. What is the row information for the repeat **user_id**? 

In [13]:
df2[df2['user_id'] == 773192]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [14]:
df2[['user_id']].user_id.nunique()

290584

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [15]:
indexToDrop = df2[df2['user_id'].duplicated()].index
indexToDrop[0]

2893

In [16]:
df2.drop(index=indexToDrop[0], inplace=True)
df2.head()

/anaconda3/envs/test_env_v03/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


Check that only one row was deleted:

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290584 entries, 0 to 294477
Data columns (total 5 columns):
user_id         290584 non-null int64
timestamp       290584 non-null object
group           290584 non-null object
landing_page    290584 non-null object
converted       290584 non-null int64
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


In [18]:
df2[['user_id']].user_id.nunique()

290584

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [19]:
num_users_df2 = df2[['user_id']].user_id.nunique()
num_converted = df2[df2['converted'] == 1].shape[0]

prop_converted_all = num_converted / num_users_df2
prop_converted_all

0.11959708724499628

In [20]:
# another way to calculate the probability:
df2['converted'].mean()

0.11959708724499628

b. Given that an individual was in the `control` group, what is the probability they converted?

In [21]:
df2_contr = df2[df2['group'] == 'control']

# single line version of probability equations above
prop_converted_contr = df2_contr[df2_contr['converted'] == 1].shape[0] / df2_contr[['user_id']].user_id.nunique()
prop_converted_contr

0.1203863045004612

In [22]:
# another way to calculate the probability:
df2_contr['converted'].mean()

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [23]:
df2_treat = df2[df2['group'] == 'treatment']

num_users_df2_treat = df2_treat[['user_id']].user_id.nunique()
num_converted_df2_treat = df2_treat[df2_treat['converted'] == 1].shape[0]

prop_converted_treat = num_converted_df2_treat / num_users_df2_treat

# # single line version of probability equations above
# prop_converted_treat = df2_treat[df2_treat['converted'] == 1].shape[0] / df2_treat[['user_id']].user_id.nunique()
prop_converted_treat

0.11880806551510564

In [24]:
# another way to calculate the probability:
df2_treat['converted'].mean()

0.11880806551510564

d. What is the probability that an individual received the new page?

In [25]:
df2_newPg = df2[df2['landing_page'] == 'new_page']

df2_newPg['user_id'].count() / df2['user_id'].count()

0.5000619442226688

e. Consider your results from a. through d. above, and explain below whether you think there is sufficient evidence to say that the new treatment page leads to more conversions.

**Your answer goes here.**

In [26]:
# FROM UDACITY
#   Section 3 Practical Statistics
#   Lesson 11: Confidence Intervals
#   Part 3 - Screencast: Sampling Distributions and Confidence Intervals
# ======================================================================

# import statsmodels.stats.api as sms
# 
# X1 = coffee_red[coffee_red['drinks_coffee'] == True]['height']
# X2 = coffee_red[coffee_red['drinks_coffee'] == False]['height']
# 
# cm = sms.CompareMeans(sms.DescrStatsW(X1), sms.DescrStatsW(X2))
# cm.tconfint_diff(usevar='unequal')


<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

**Put your answer here.**

* The null hypothesis is that the conversion rate for the old page is greater than or equal to the conversion rate of the new page.
* The alternative hypothesis is that the conversion rate for the old page is less than the conversion rate of the new page.
* Expressions for the null ($H_0$) and alternatuve hypotheses ($H_1$) are as follows:

$$H_0: p_{old} \ge p_{new}$$
$$H_1: p_{old} < p_{new}$$

* Another way to write the null and alternative hypothesis expressions:

$$H_0: p_{old} - p_{new} \ge 0$$
$$H_1: p_{old} - p_{new} < 0$$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br>
* That is: $p_{new} = p_{old} = p_{regardless\_of\_page} $ <br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br>
* That is:
  * The sample size for old page ($n_{old}$) == the number of 'old_page' entries in ab_data.csv
  * The sample size for old page ($n_{new}$) == the number of 'new_page' entries in ab_data.csv

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br>

a. What is the **convert rate** for $p_{new}$ under the null? 

* The null assumes that $p_{new} = p_{old} = p_{regardless\_of\_page} $ <br><br>
* And $p_{regardless\_of\_page} $ == <code>df2['converted'].mean() </code>

In [27]:
df2['converted'].mean()

0.11959708724499628

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [28]:
df2['converted'].mean()

0.11959708724499628

c. What is $n_{new}$?

In [29]:
n_new = df2.where(df['landing_page'] == 'new_page').landing_page.count()
n_new

145310

* For reference, here is the dataframe grouped by landing_page:

In [30]:
df2.groupby('landing_page').count()

,user_id,timestamp,group,converted
landing_page,,,,
new_page,145310,145310,145310,145310
old_page,145274,145274,145274,145274


d. What is $n_{old}$?

In [31]:
n_old = df2.where(df['landing_page'] == 'old_page').landing_page.count()
n_old

145274

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [32]:
new_page_converted = np.random.choice([1,0], n_new, p=[df2.converted.mean(), (1-df2.converted.mean())])

* **Notional code, v01:** <code> new_page_converted = df2.sample(n_new, replace=True) </code>

* **Notional code, v02:**<br>
<code>#simulate under null for n_new
new_page_converted = np.random.choice([1, 0], size=len(df2_treatment.index), p=[df2.converted.mean(), (1-(df2.converted.mean()))])</code><br>
   * [https://github.com/thomasdmtang/Udacity-A-B-Test/blob/master/Analyze_ab_test_results_notebook.ipynb]

* **Notional code, v03:**<br>
<code>new_page_converted = np.random.choice([0,1],N_new, p=(p_new,1-p_new))</code><br>
   * [https://github.com/Alicelibinguo/Analyzing-Website-Landing-Page-A-B-Test-Results-/blob/master/Analyze_ab_test_results_notebook.ipynb]

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [33]:
old_page_converted = np.random.choice([1,0], n_old, p=[df2.converted.mean(), (1-df2.converted.mean())])

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [36]:
new_page_converted.mean() - old_page_converted.mean()

-2.2241538280245998e-05

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in a numpy array called **p_diffs**.

In [37]:
# p_diffs = []
# prob_both_pages = df2.converted.mean()
# for _ in range(10000):
#     new_page_converted = np.random.choice([1,0], n_new, p=[prob_both_pages, (1-prob_both_pages)])
#     old_page_converted = np.random.choice([1,0], n_old, p=[prob_both_pages, (1-prob_both_pages)])
#     p_diffs.append(new_page_converted.mean() - old_page_converted.mean())

In [38]:
new_page_converted = np.random.binomial(n_new, df2.converted.mean(),  10000)/n_new
old_page_converted = np.random.binomial(n_old, df2.converted.mean(),  10000)/n_old
p_diffs = new_page_converted - old_page_converted

In [ ]:
# FROM UDACITY:
# 08_Sampling Distributions
# Sampling Distributions & Confidence Intervals Screencast.ipynb
# ========================================================
# 
# bootsample = coffee_red.sample(200, replace=True)
# 
# boot_means = []
# for _ in range(10000):
#     bootsample = coffee_red.sample(200, replace=True)
#     boot_means.append(bootsample[bootsample['drinks_coffee'] == True]['height'].mean())

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

k. In words, explain what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**Put your answer here.**

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = 
convert_new = 
n_old = 
n_new = 

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**Put your answer here.**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Put your answer here.**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

**Put your answer here.**

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**Put your answer here.**

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')

In [ ]:
### Create the necessary dummy variables

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
### Fit Your Linear Model And Obtain the Results

<a id='conclusions'></a>
## Conclusions

Congratulations on completing the project! 

### Gather Submission Materials

Once you are satisfied with the status of your Notebook, you should save it in a format that will make it easy for others to read. You can use the __File -> Download as -> HTML (.html)__ menu to save your notebook as an .html file. If you are working locally and get an error about "No module name", then open a terminal and try installing the missing module using `pip install <module_name>` (don't include the "<" or ">" or any words following a period in the module name).

You will submit both your original Notebook and an HTML or PDF copy of the Notebook for review. There is no need for you to include any data files with your submission. If you made reference to other websites, books, and other resources to help you in solving tasks in the project, make sure that you document them. It is recommended that you either add a "Resources" section in a Markdown cell at the end of the Notebook report, or you can include a `readme.txt` file documenting your sources.

### Submit the Project

When you're ready, click on the "Submit Project" button to go to the project submission page. You can submit your files as a .zip archive or you can link to a GitHub repository containing your project files. If you go with GitHub, note that your submission will be a snapshot of the linked repository at time of submission. It is recommended that you keep each project in a separate repository to avoid any potential confusion: if a reviewer gets multiple folders representing multiple projects, there might be confusion regarding what project is to be evaluated.

It can take us up to a week to grade the project, but in most cases it is much faster. You will get an email once your submission has been reviewed. If you are having any problems submitting your project or wish to check on the status of your submission, please email us at dataanalyst-project@udacity.com. In the meantime, you should feel free to continue on with your learning journey by beginning the next module in the program.